<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/10_neural_machine_translation_with_attention(example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! python -m spacy download fr

## 10-01 準備

### 必要なモジュール、パッケージ等をインポート

In [ ]:
import sys
import random
import math
import time
from datetime import datetime

import numpy as np
import spacy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

### 再現性確保のための設定

In [ ]:
SEED = 123

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

In [ ]:
spacy.load("fr")
spacy.load("en")

### torchtext.dataのフィールドを作成
* 翻訳元のテキストと、翻訳先のテキストそれぞれについて、フィールドを作成。
* tokenizationはいずれもspaCyにする。
* init_tokenとeos_tokenを指定する。
* 大文字はすべて小文字にしてしまう。
 * 本格的な翻訳システムを作る場合は、大文字を小文字にするのはNG。

In [ ]:
SRC = Field(tokenize="spacy",
            init_token="<sos>",
            eos_token="<eos>",
            lower=True)

TRG = Field(tokenize="spacy",
            init_token="<sos>",
            eos_token="<eos>",
            lower=True)

### 訓練データ、検証データ、テストデータを用意する
* 今回は、Multi30k Data Repositoryというデータセットから、仏英翻訳の部分を使う。 
 * https://github.com/multi30k/dataset

In [ ]:
Multi30k.download('.')

* フランス語のデータはgithubから手動でダウンロードし、解凍。

In [ ]:
!wget https://github.com/multi30k/dataset/raw/master/data/task1/raw/train.fr.gz
!gunzip train.fr.gz
!mv train.fr ./multi30k/
!wget https://github.com/multi30k/dataset/raw/master/data/task1/raw/val.fr.gz
!gunzip val.fr.gz
!mv val.fr ./multi30k/

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(path='./multi30k/', exts=('.fr', '.en'), fields=(SRC, TRG))

In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

In [ ]:
print(vars(train_data.examples[0]))

In [ ]:
print(train_data.examples[0].src)

In [ ]:
print(train_data.examples[0].trg)

### 語彙を構築する
* 出現頻度が1回の単語はすべて未知語としてまとめてしまう。

In [ ]:
SRC.build_vocab(train_data, min_freq=1)
TRG.build_vocab(train_data, min_freq=1)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

* 語彙のうちID順で最初の２つは、やはり特殊な単語になっている。
 * `<unk>`は未知語
 * `<pad>`はパディング用の単語

In [ ]:
print(SRC.vocab.itos[:10])
print(TRG.vocab.itos[:10])

### デバイスを設定する
* GPUが使えるなら使う。

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### ミニバッチを取り出すイテレータを作る
* ミニバッチのshapeは、[単語列長, ミニバッチの大きさ]となる。
 * 正確には、単語列長は、ミニバッチに含まれる単語列のうち最長のものの長さ、である。
 * 最長の単語列以外の単語列は、最長の列と同じ長さになるまで、末尾にパディング用の特殊なトークンで埋められる。

In [ ]:
BATCH_SIZE = 100

train_iterator = BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.src))
valid_iterator = BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

## 10-02　モデル
* エンコーダ
 * 入力として与えられた単語列をまず埋め込み、そしてRNNによって隠れ状態を表すベクトルへと変換する。
 * 今回は、bidirectionalなRNNを使う。
* デコーダ
 * エンコーダRNNから隠れ状態ベクトルなどの情報を受け取って、翻訳先の言語の単語列を、RNNによって生成する。
* エンコーダ、デコーダ両方とも、LSTMではなく、GRUを使う。

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim,
               enc_hid_dim, dec_hid_dim, p=0.5):
    super().__init__()

    self.input_dim = input_dim
    self.emb_dim = emb_dim
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim

    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # srcの形は[単語列長, バッチサイズ]

    embedded = self.dropout(self.embedding(src))
    # embeddedの形は[単語列長, バッチサイズ, 埋め込み次元数]

    outputs, hidden = self.rnn(embedded)
    # outputsの形は[単語列長, バッチサイズ, encoderの隠れ状態の次元数*2]
    # hiddenの形は[レイヤー数*2, バッチサイズ, encoderの隠れ状態の次元数]
    #   hiddenは[forward_1, backward_1, forward_2, backward_2, ...]とスタックされている。

    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
    # hiddenの型が[バッチサイズ, デコーダの隠れ状態の次元数]に変わる

    return outputs, hidden

### 注意機構(attention)
* attentionの部分だけでクラスを定義する。
* decoder側の各トークンについて、encoderの出力ベクトル列を線型結合するための重みを求める。
 * encoder側のトークンのうち、関連度の高いトークンほど、重みが大きくなる。

In [ ]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim, attn_dim):
    super().__init__()

    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.attn_in = (enc_hid_dim * 2) + dec_hid_dim
    self.attn = nn.Linear(self.attn_in, attn_dim)

  def forward(self, decoder_hidden, encoder_outputs):
    # 翻訳元のシーケンス長を取得
    src_len = encoder_outputs.shape[0]

    # decoder_hiddenの形は[バッチサイズ, decoderの隠れ状態次元数]
    # unsqueezeで軸を真ん中に追加し、その軸の方向にsrc_len回、同じ内容を繰り返す
    repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
    # これにより、decoder_hiddenの形は
    # [バッチサイズ, 翻訳元のシーケンス長, decoderの隠れ状態次元数]となる

    # 上で作ったrepeated_decoder_hiddenと合うように、encoder_outputsの軸を入れ替える
    # encoder_outputsの元の形は[翻訳元のシーケンス長, バッチサイズ, encoderの隠れ状態次元数*2]
    # 軸を入れ替えると、[バッチサイズ, 翻訳元のシーケンス長, encoderの隠れ状態次元数*2]という形に変わる
    # これでrepeated_decoder_hiddenと合う形になる
    encoder_outputs = encoder_outputs.permute(1, 0, 2)

    # attentionのenergyの計算
    # ここで使っているのは内積ではない
    # ベクトルをconcatenateして、全結合層self.attnに入力し、tanhで-1~1の範囲に収めている
    energy = torch.tanh(self.attn(torch.cat((
        repeated_decoder_hidden,
        encoder_outputs),
        dim = 2)))

    # energyの形は[バッチサイズ, シーケンス長, attentionの次元数]
    # attentionの次元数方向に和をとって[バッチサイズ, 翻訳元のシーケンス長]という形にする
    attention = torch.sum(energy, dim=2)
    # attentionの各エントリが、翻訳元の各トークンとの"関連度"を表している

    # attentionの形は[バッチサイズ, シーケンス長]
    # シーケンス長方向にsoftmax関数を適用（これにより線型結合に使う係数が得られる）
    return F.softmax(attention, dim=1)

### デコーダ
* やはり、LSTMではなく、GRUを使う。
* 単語列を生成できる必要があるため、bidirectionalなものは使えない。
* デコーダがエンコーダから受け取る情報として、エンコーダRNNの最後の隠れ状態を使うだけではあまりうまくいかない。
* そこで今回は、エンコーダRNNの隠れ状態だけでなく、エンコーダRNNの出力ベクトル列（入力単語列に沿って得られるすべて隠れ状態）も、デコーダの入力の一部として使う。
 * エンコーダRNNの出力ベクトル列を表すテンソルの形は[単語列長, バッチサイズ, 隠れ状態の次元数*2]である。
 * これらの重み付きの和を、attentionを利用して求め、デコーダへの入力の一部として利用する。

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim,
               enc_hid_dim, dec_hid_dim,
               attention, p=0.5):
    super().__init__()

    self.emb_dim = emb_dim
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.output_dim = output_dim
    self.attention = attention

    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
    self.out = nn.Linear(dec_hid_dim + (enc_hid_dim * 2) + emb_dim, output_dim)
    self.dropout = nn.Dropout(p=p)

  # Attentionのために追加で必要となる関数
  # decoder側の個々のトークンについて別々に呼び出される
  def _weighted_encoder_rep(self, decoder_hidden, encoder_outputs):
    a = self.attention(decoder_hidden, encoder_outputs)
    # 戻ってきたaの形は[バッチサイズ, シーケンス長]
    # このaの形を、軸を真ん中に追加することで、[バッチサイズ, 1, シーケンス長]に変える
    a = a.unsqueeze(1)

    # repeated_decoder_hiddenと合うように、encoder_outputsの軸を入れ替える
    # encoder_outputsの元の形は[シーケンス長, バッチサイズ, encoderの隠れ状態次元数*2]
    # 軸を入れ替えると、[バッチサイズ, シーケンス長, encoderの隠れ状態次元数*2]という形に変わる
    encoder_outputs = encoder_outputs.permute(1, 0, 2)

    # aを重みとした、encoderの出力ベクトル列の線型結合を求める
    weighted_encoder_rep = torch.bmm(a, encoder_outputs)
    # weighted_encoder_repの形は[バッチサイズ, 1, encoderの隠れ状態次元数*2]
    weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
    # weighted_encoder_repの形は[1, バッチサイズ, encoderの隠れ状態次元数*2]になる
    return weighted_encoder_rep

  # forwardメソッドは、decoder側の個々のトークンについて、別々に呼び出される
  def forward(self, input, decoder_hidden, encoder_outputs):
    # inputの形は[バッチサイズ]
    input = input.unsqueeze(0)
    # inputの形が[1, バッチサイズ]になる

    embedded = self.dropout(self.embedding(input))
    # embeddedの形は[1, バッチサイズ, 埋め込み次元数]

    # encoderの出力ベクトル列の線型結合を求める（つまりattentionの計算）
    weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                      encoder_outputs)
    # 戻ってきたweighted_encoder_repの形は[1, バッチサイズ, encoderの隠れ状態次元数*2]
    # encoderの出力ベクトル列の線型結合と、埋め込みベクトルをconcatenateする
    rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

    # 次のトークンを予測する
    output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))
    embedded = embedded.squeeze(0)
    # embeddedの形は[バッチサイズ, 埋め込み次元数]になる
    output = output.squeeze(0)
    # outputの形は[バッチサイズ, decoderの隠れ状態次元数]になる
    weighted_encoder_rep = weighted_encoder_rep.squeeze(0)
    # weighted_encoder_repの形は[バッチサイズ, encoderの隠れ状態次元数*2]になる
    output = self.out(torch.cat((output, weighted_encoder_rep, embedded), dim = 1))

    return output, decoder_hidden.squeeze(0)

### エンコーダとデコーダをまとめる
* エンコーダとデコーダをまとめて扱うためのクラスを定義する。

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
       
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio=0.5, no_length_limit=False):
    # trgの形は[単語列長, バッチサイズ]
    max_len, batch_size = trg.shape
    if no_length_limit:
      max_len *= 2
    trg_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

    encoder_outputs, hidden = self.encoder(src)

    # デコーダへの最初の入力トークンは<sos>トークン
    output = trg[0,:]
    for t in range(1, max_len):
      output, hidden = self.decoder(output, hidden, encoder_outputs)
      outputs[t] = output
      teacher_force = torch.rand(1).item() < teacher_forcing_ratio
      top1 = output.max(1)[1]
      output = (trg[t] if teacher_force else top1)

    return outputs

### 定数の定義

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 128
DEC_HID_DIM = 128
ATTN_DIM = 16
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

### モデルのインスタンスを作る

In [ ]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

model = Seq2Seq(
    Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, p=ENC_DROPOUT),
    Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, attn, p=DEC_DROPOUT),
    device
    ).to(device)

In [ ]:
!pip install torch-summary

In [ ]:
from torchsummary import summary

In [ ]:
summary(model)

### optimizerの準備
* Adamを使う

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

### 損失関数

In [ ]:
PAD_IDX = TRG.vocab.stoi['<pad>']
print(PAD_IDX)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

## 10-03 学習や評価を補助するための関数

### 訓練のための関数

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0
  for batch in iterator:
    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()
    output = model(src, trg)
    output = output[1:].view(-1, output.shape[-1])
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

### 評価のための関数

In [ ]:
def evaluate(model, iterator, criterion):
  model.eval()

  epoch_loss = 0
  with torch.no_grad():
    for batch in iterator:
      src = batch.src
      trg = batch.trg

      output = model(src, trg, 0) # teacher forcingは無効にする
      output = output[1:].view(-1, output.shape[-1])
      trg = trg[1:].view(-1)

      loss = criterion(output, trg)
      epoch_loss += loss.item()

  return epoch_loss / len(iterator), output

### 経過時間を表示する関数

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

### チェックポイントの保存・読み込みの関数

In [ ]:
def save_checkpoint(path, model, optimizer, epoch):
  torch.save({
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'epoch': epoch,
      }, path)
  
def load_checkpoint(path, model, optimizer):
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  return checkpoint['epoch']

## 10-04 学習の実行


* 途中まで学習済みのモデルがあればロードする。
 * チェックポイントを保存するパスは各自設定してください。

In [ ]:
# 下記のパスは、あくまでも、例です。
LOAD_PATH = '/content/drive/MyDrive/2020Courses/NLP/10_NMT_fr.tar' 
SAVE_PATH = '/content/drive/MyDrive/2020Courses/NLP/10_NMT_fr.tar'

# チェックポイントのロード（エラーは無視）
init_epoch = 0
try:
  init_epoch = load_checkpoint(LOAD_PATH, model, optimizer)
except:
  print("No checkpoints.")
  pass
init_epoch += 1


* trainingのループ

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(init_epoch, init_epoch + N_EPOCHS):

  start_time = time.time()
  train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
  valid_loss, _ = evaluate(model, valid_iterator, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if epoch % 10 == 0:
    save_checkpoint(SAVE_PATH, model, optimizer, epoch)

  print(f'Epoch: {epoch:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

## 10-05 テストセット上で評価

### 定量的評価

In [ ]:
test_loss, output = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f}\t|\tTest PPL: {math.exp(test_loss):7.3f} |')

### 定性的評価

In [ ]:
load_checkpoint(LOAD_PATH, model, optimizer)

In [ ]:
def iseq2sseq(field, seq):
  return [field.vocab.itos[i] for i in seq if i != field.vocab.stoi[field.pad_token] and i != field.vocab.stoi[field.eos_token]]

In [ ]:
test_batch = list(test_iterator)[0]
outputs = model(test_batch.src, test_batch.trg, 0, True)
for j in range(test_batch.src.shape[1]):
  print('>', ' '.join(iseq2sseq(SRC, test_batch.src[1:, j])))
  print('=', ' '.join(iseq2sseq(TRG, test_batch.trg[1:, j])))
  print('<', ' '.join(iseq2sseq(TRG, outputs[1:, j, :].squeeze().argmax(1))))
  print('-' * 64)